In [123]:
import yfinance as yf
from scipy.stats import norm, zscore
import numpy as np
import pandas as pd
import math
import scipy as sq
import yahoo_fin.stock_info as si
from datetime import datetime, timedelta, date
import os


def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
ticker = yf.Ticker("ASO")
todays_data = ticker.history(period='1y')
todays_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-06-08,40.165678,40.395196,37.281733,39.566936,5269100,0.0,0
2021-06-09,39.596870,40.884168,39.277540,39.816410,2728500,0.0,0
2021-06-10,39.786472,40.944041,39.337414,39.946136,3061300,0.0,0
2021-06-11,40.335317,41.652556,40.115781,41.622616,2785300,0.0,0
2021-06-14,41.752345,41.896044,40.514941,41.602657,1761500,0.0,0
...,...,...,...,...,...,...,...
2022-06-02,33.840000,36.000000,33.759998,35.750000,2686900,0.0,0
2022-06-03,35.230000,35.910000,34.459999,34.730000,2341600,0.0,0
2022-06-06,35.049999,35.599998,34.009998,35.570000,2556500,0.0,0


In [3]:
def get_current_price(symbol):
    tk = yf.Ticker(symbol)
    return tk.history(period='1d')['Close'][0]

def get_days_to_expiration(symbol):
    tk = yf.Ticker(symbol)
    exps = tk.options
    e = exps[0]
    dt_obj = datetime.strptime(e, '%Y-%m-%d')
    days_left = abs((datetime.now() - dt_obj).days)
    return days_left

def options_chain(symbol):
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    e = exps[0] 

    opt = tk.option_chain(e)
    calls = pd.DataFrame(opt.calls)
    puts = pd.DataFrame(opt.puts)
    calls['expirationDate'] = e
    puts['expirationDate'] = e
    
    options = pd.concat([options, calls])
    options = pd.concat([options, puts])
    
    options['CALL'] = options['contractSymbol'].str[4:].apply(lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark_price'] = (options['bid'] + options['ask']) / 2
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    
#     vol = []
#     for row in options.iterrows():
#         S = get_current_price(symbol)
#         X = row[1].strike
#         r = 0.03
#         T = get_days_to_expiration(symbol)
#         option = row[1].mark_price
#         p_c = 'C'
#         if row[1].CALL == False :
#             p_c = 'P'
            
#         IV = Calculate_IV_Call_Put(S, X, r, T, option, p_c)
#         vol.append(IV)
    
#     options["Black Scholes IV"] = vol
    return options

In [121]:
def get_expected_move(symbol):
    stock = yf.Ticker(symbol)
    curr = stock.history(period='1d')['Close'][0]
    options = options_chain(symbol)
        
    test_list = list(set(options.strike))
    temp = sorted(test_list, key=lambda x:abs(x-curr))
    if len(temp) < 3:
        return "Error"
    
    minVal = temp[0]
    higherVal = temp[1] if temp[1] > temp[2] else temp[2]
    lowerVal = temp[2] if temp[1] > temp[2] else temp[1]
    
    strad = options.loc[options.strike == minVal]
    low_stran = options.loc[options.strike == lowerVal]
    high_stran = options.loc[options.strike == higherVal]
    
    # IV Expected Move
    expected_move = curr * strad.impliedVolatility * math.sqrt(get_days_to_expiration(symbol)/365)
    per = expected_move / curr
#     print("Stock Expected Move based on IV is around -/+ %.5f" % expected_move.mean(), "or %.5f" % (per.mean() * 100), "%")
    # Expected Move = Stock Price x (Implied Volatility / 100) x square root of (Days to Expiration / 365)
    
    straddle_sum = 0
    if len(strad.mark_price) == 2:
        straddle_sum = strad.mark_price.sum()
    elif len(strad.mark_price) == 1:
        straddle_sum = strad.mark_price.sum() * 2

    strangle_sum = 0
    high_mark = high_stran.loc[high_stran.CALL == True].mark_price
    low_mark = low_stran.loc[low_stran.CALL == False].mark_price
    if not high_mark.empty and not low_mark.empty:
        strangle_sum = high_mark.iloc[0] + low_mark.iloc[0]
    
    if straddle_sum == 0:
        if len(strad.lastPrice) == 2:
            straddle_sum = strad.lastPrice.sum()
        elif len(strad.lastPrice) == 1:
            straddle_sum = strad.lastPrice.sum() * 2
    if strangle_sum == 0:
        high_last = high_stran.loc[high_stran.CALL == True].lastPrice
        low_last = low_stran.loc[low_stran.CALL == False].lastPrice
        if not high_last.empty and not low_last.empty:
            strangle_sum = high_last.iloc[0] + low_last.iloc[0]

    #Straddle Expected Move - 70% of ATM straddle + 30% of 1st Strangle
    total = 0.7 * (straddle_sum) + 0.3 * (strangle_sum)
    
#     print("Stock Expected Move based on Straddles is around -/+ %.5f" % result.mark_price.sum(), "or %.5f" % (straddle * 100), "%")
    # IV Expected, ATM Straddle, ATM Straddle + 1st Strangle, Avg of IV + ATM
    return (per.mean(), straddle_sum / curr, total / curr, (per.mean() + straddle_sum / curr) / 2)

get_expected_move("SIG")

(0.1245168856733325,
 0.12138263516708037,
 0.08496784461695624,
 0.12294976042020643)

In [155]:
def get_put_options(symbol, price_ceiling = -1):
    tk = yf.Ticker(symbol)
    exps = tk.options
    e = exps[0] 

    opt = tk.option_chain(e)
    puts = pd.DataFrame(opt.puts)
    
    curr_price = tk.history(period='1d')['Close'][0]

    puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    puts.insert(6, 'mark', (puts['bid'] + puts['ask']) / 2)
    puts.insert(12, 'dist%', ((curr_price - puts['strike'] ) / puts['strike']) * 100)
    
    puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'inTheMoney'])
#     temp = puts.pop('contractSymbol') 
#     puts['contractSymbol'] = temp

    
    moves = get_expected_move(symbol)
    puts['average%'] = moves[3] * 100
    puts['IV_move%'] = moves[0] * 100
    puts['straddle_move%'] = moves[1] * 100
    puts['strad_strang_move%'] = moves[2] * 100
    
    puts['expirationDate'] = e
    
    if price_ceiling == -1:
        price_ceiling = curr_price
    
    index_names = puts[(puts['strike'] > price_ceiling)].index
    puts = puts.drop(index_names)    
    puts.sort_values(by=['strike'], inplace=True, ascending=False)
    puts.reset_index(drop=True, inplace=True)
    
    
    zscore = (puts['dist%'] / puts['average%']) 
    puts.insert(6, 'probability_profit%', norm.cdf(zscore) * 100)
    puts.insert(7, '%gain_of_collateral', (puts['mark'] * 100 / puts['strike']))    
    return puts

get_put_options("AAPL")

,contractSymbol,strike,lastPrice,bid,ask,mark,probability_profit%,%gain_of_collateral,volume,openInterest,impliedVolatility,dist%,average%,IV_move%,straddle_move%,strad_strang_move%,expirationDate
0,AAPL220610P00147000,147.0,1.19,1.14,1.21,1.175,61.630018,0.799320,24567.0,7036.0,0.308601,0.653066,2.207959,2.246414,2.169505,1.98601,2022-06-10
1,AAPL220610P00146000,146.0,0.85,0.82,0.86,0.840,72.841093,0.575342,17966.0,9436.0,0.312019,1.342470,2.207959,2.246414,2.169505,1.98601,2022-06-10
2,AAPL220610P00145000,145.0,0.59,0.57,0.61,0.590,82.240176,0.406897,24849.0,23650.0,0.319831,2.041384,2.207959,2.246414,2.169505,1.98601,2022-06-10
3,AAPL220610P00144000,144.0,0.41,0.40,0.42,0.410,89.352527,0.284722,9753.0,9255.0,0.326179,2.750005,2.207959,2.246414,2.169505,1.98601,2022-06-10
4,AAPL220610P00143000,143.0,0.28,0.26,0.28,0.270,94.189984,0.188811,9037.0,6742.0,0.331550,3.468536,2.207959,2.246414,2.169505,1.98601,2022-06-10
5,AAPL220610P00142000,142.0,0.20,0.18,0.20,0.190,97.134475,0.133803,7157.0,6662.0,0.344733,4.197188,2.207959,2.246414,2.169505,1.98601,2022-06-10
6,AAPL220610P00141000,141.0,0.14,0.12,0.14,0.130,98.731191,0.092199,5452.0,4656.0,0.356452,4.936175,2.207959,2.246414,2.169505,1.98601,2022-06-10
7,AAPL220610P00140000,140.0,0.10,0.09,0.10,0.095,99.498946,0.067857,13716.0,21142.0,0.370123,5.685719,2.207959,2.246414,2.169505,1.98601,2022-06-10
8,AAPL220610P00139000,139.0,0.07,0.05,0.09,0.070,99.824681,0.050360,2433.0,5789.0,0.400397,6.446048,2.207959,2.246414,2.169505,1.98601,2022-06-10
9,AAPL220610P00138000,138.0,0.05,0.04,0.05,0.045,99.945999,0.032609,1097.0,3745.0,0.394537,7.217396,2.207959,2.246414,2.169505,1.98601,2022-06-10


In [33]:
# Get Earnings Price Effect
# Returns Pandas DataFrame with Data of Market Close to Next Day Open and Close
def get_earnings_price_effect(symbol, period = "10y"):
    stock = yf.Ticker(symbol)
    hist = stock.history(period)
    earn_hist = si.get_earnings_history(symbol)
    data = {}
    is_AMC = earn_hist[0]['startdatetimetype'] == "AMC" #Default AMC
    for earning in earn_hist: 
        earn_date = earning['startdatetime']
        dt_obj = datetime.strptime(earn_date, '%Y-%m-%dT%H:%M:%S.000Z')
        date = dt_obj.date()
        # If hour is < 9, count as BMO, else count as AMC
        if(dt_obj.hour < 9): 
            is_AMC = False
        else:
            is_AMC = True

        if is_AMC:
            earning_day = hist.loc[(hist.index == str(date))]
            earning_next_day = hist.loc[hist.index == str(date + timedelta(days = 1))]
            if not earning_day.empty and not earning_next_day.empty:
                earning_effect_close_to_next_open = earning_next_day["Open"].iloc[0] - earning_day["Close"].iloc[0]
                earning_effect_close_to_next_close = earning_next_day["Close"].iloc[0] -  earning_day["Close"].iloc[0]

                data[str(date)] = [earning_effect_close_to_next_open, 
                                  earning_effect_close_to_next_open * 100 / earning_day["Close"].iloc[0] ,
                                   earning_effect_close_to_next_close ,
                                  earning_effect_close_to_next_close * 100 / earning_day["Close"].iloc[0]]

        else:
            earning_day = hist.loc[(hist.index == str(date))]
            earning_prev_day = hist.loc[hist.index == str(date - timedelta(days = 1))]
            if not earning_day.empty and not earning_prev_day.empty:
                earning_effect_prev_close_to_open = earning_day["Open"].iloc[0] -  earning_prev_day["Close"].iloc[0]
                earning_effect_prev_close_to_close = earning_day["Close"].iloc[0] -  earning_prev_day["Close"].iloc[0]

                data[str(date)] = [earning_effect_prev_close_to_open,
                                  earning_effect_prev_close_to_open * 100/ earning_prev_day["Close"].iloc[0] ,
                                   earning_effect_prev_close_to_close,
                                  earning_effect_prev_close_to_close * 100/ earning_prev_day["Close"].iloc[0]]

    earn_data = pd.DataFrame.from_dict(data, orient='index',
                columns=['day_close_to_next_day_open', '%change_close_to_next_open',
                        'day_close_to_next_day_close', '%change_close_to_next_close'])
    
    new_row = pd.DataFrame({'day_close_to_next_day_open':earn_data['day_close_to_next_day_open'].abs().mean(), '%change_close_to_next_open':earn_data['%change_close_to_next_open'].abs().mean(), 
                            'day_close_to_next_day_close':earn_data['day_close_to_next_day_close'].abs().mean(), '%change_close_to_next_close':earn_data['%change_close_to_next_close'].abs().mean()}, 
                           index=["Absolute Average"])

    earn_data = pd.concat([earn_data, new_row])
    earn_data.insert(0, 'Symbol', symbol)

    return earn_data

get_earnings_price_effect("FB")

,Symbol,day_close_to_next_day_open,%change_close_to_next_open,day_close_to_next_day_close,%change_close_to_next_close
2022-04-27,FB,27.970001,15.987426,30.779999,17.593598
2022-02-02,FB,-78.350006,-24.256968,-85.240005,-26.390095
2021-10-25,FB,-0.429993,-0.130820,-12.880005,-3.918587
2021-07-28,FB,-12.279999,-3.289755,-14.959991,-4.007713
2021-04-28,FB,23.019989,7.495926,22.410004,7.297298
2021-01-27,FB,5.039978,1.851980,-7.140015,-2.623655
2020-10-29,FB,-6.329987,-2.254028,-17.720001,-6.309868
2020-07-30,FB,21.320007,9.091688,19.169998,8.174839
2020-04-29,FB,12.729996,6.555433,10.520004,5.417377
2020-01-29,FB,-16.699997,-7.481072,-13.699997,-6.137167


In [134]:
indexOptions = pd.read_csv('cboesymboldirequityindex.csv')
weeklyOptions = pd.read_csv('cboesymboldirweeklys.csv')

indexOptions.drop(columns = [' DPM Name', ' Post/Station'], inplace=True)
indexOptions.columns = ['name', 'symbol']
weeklyOptions.columns = ['name', 'symbol']
optionSet = set(indexOptions['symbol'])
weeklySet = set(weeklyOptions['symbol'])

In [88]:
def get_companies_with_earnings_today_AMC_or_tomm_BMO(add_days = 0):
    curr_date = datetime.now().date() +  timedelta(add_days)
        
    earn_today = si.get_earnings_for_date(str(curr_date))
    earn_tomm = si.get_earnings_for_date(str(curr_date + timedelta(1)))
    comp_today = pd.DataFrame.from_dict(earn_today)
    comp_tomm = pd.DataFrame.from_dict(earn_tomm)

    index_names = comp_today[(comp_today['startdatetimetype'] != 'AMC')].index
    comp_today = comp_today.drop(index_names)

    index_names = comp_tomm[(comp_tomm['startdatetimetype'] != 'BMO')].index
    comp_tomm = comp_tomm.drop(index_names)

    comp = pd.DataFrame()
    comp = pd.concat([comp_today, comp_tomm])
    comp = comp.drop(columns = ['gmtOffsetMilliSeconds', 'quoteType', 'epsactual', 'epssurprisepct'])
    comp = comp.drop_duplicates()
    comp.reset_index(drop=True, inplace=True)

    comp.insert(2, "stock_price", 0)
    comp.insert(3, "is_weekly", False)
    comp['volume'] = 0
    for row in comp.iterrows():
        ticker = row[1].ticker
        if ticker not in optionSet:
            continue

        tk = yf.Ticker(ticker)
        exps = tk.options        
        if not exps: 
            continue

        e = exps[0]
        opt = tk.option_chain(e)
        volumeSum = opt.calls.volume.sum() + opt.puts.volume.sum()
        
        if volumeSum < 5000:
            continue
            
        openInterestSum = opt.calls.openInterest.sum() + opt.puts.openInterest.sum()

        rowVal = comp.index[comp['ticker'] == ticker]
        comp.loc[rowVal, 'stock_price'] = round(tk.history(period='1d')['Close'][0] , 2)
        comp.loc[rowVal, 'volume'] = volumeSum
        comp.loc[rowVal, 'openInterest'] = openInterestSum
        comp.loc[rowVal, 'is_weekly'] = (ticker in weeklySet)
        
#         move = get_expected_move(ticker)
#         comp.loc[rowVal, 'IV_expected_move%'] = move[0] * 100
#         comp.loc[rowVal, 'straddle_expected_move%'] = move[1] * 100        
#         comp.loc[rowVal, 'straddle_strangle_expected_move%'] = move[2] * 100        

    index_names = comp[(comp['volume'] == 0) | (comp['stock_price'] <= 5)].index
    comp = comp.drop(index_names)

    comp.sort_values(by=['volume'], inplace=True, ascending=False)
    comp.reset_index(drop=True, inplace=True)

    return comp

today_comp = get_companies_with_earnings_today_AMC_or_tomm_BMO(0)
today_comp.to_html("today_companies.html")

In [135]:
def get_IV_crush_for_puts():
    dir = os.path.join(str(date.today().year), str(date.today() + timedelta(-1)))

    if not os.path.exists(dir):
        print("Directory does not exist for IV Crush")
        return
    num_files = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name))])

    for x in range(num_files - 1):
        list_companies = pd.read_csv(os.path.join(dir, "companies.csv"))
        tickers = list_companies[['ticker', 'stock_price']]

        company = pd.read_csv(os.path.join(dir, tickers['ticker'][x] + '.csv'))
        df = get_put_options(tickers['ticker'][x], tickers['stock_price'][x])[['strike','impliedVolatility']]
        company['IV_crush'] = abs(df['impliedVolatility'] - company['impliedVolatility'])/ company['impliedVolatility']
        avg = company['IV_crush'].mean()
        company.loc['Average'] = {'IV_crush': avg}
        
        company.to_csv(os.path.join(dir, tickers['ticker'][x] + '.csv'))
        
get_IV_crush_for_puts()

In [142]:
def check_today(duration = "10y"):
    today_comp = get_companies_with_earnings_today_AMC_or_tomm_BMO(0)
    total = pd.DataFrame()
    price = pd.DataFrame()
    for row in today_comp.iterrows():
        ticker = row[1].ticker
        temp = get_put_options(ticker)
        
        outdir = os.path.join(str(date.today().year), str(date.today()))

        if not os.path.exists(outdir):
            os.mkdir(outdir)

        fullname = os.path.join(outdir, ticker + ".csv")    
        temp.to_csv(fullname)
        
        today_path = os.path.join(outdir, "companies.csv")    
        today_comp.to_csv(today_path)

        total = pd.concat([total, temp], ignore_index=True)
        total.loc["Empty"] = '' 
        total.loc["Space"] = ''        

        price_effect = get_earnings_price_effect(ticker, duration)
        price_effect.loc["Empty"] = '' 
        price_effect.loc["Space"] = '' 
        price = pd.concat([price, price_effect])   
        
    return (total, price)

In [143]:
df = check_today("3y")
today_comp.to_html("today_companies.html")
df[0].to_html('today_options.html')
df[1].to_html('today_price_effect.html')
get_IV_crush_for_puts()
print("Done")

AttributeError: Can only use .str accessor with string values!